## 导入必要的库

In [54]:
%matplotlib inline

import random
import numpy as np
from math import *
from matplotlib import pyplot as plt

In [2]:
def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

## 根据样本生成词向量
这里使用map也可以，使用set也可以

In [3]:
def buildWordVec(wordLists):
    wordVec = set([])
    for words in wordLists:
        wordVec = wordVec | set(words)
    return list(wordVec)

In [4]:
dataset, datalabel = loadDataSet()
wordVec = buildWordVec(dataset)
print(wordVec)

['dog', 'please', 'I', 'park', 'quit', 'not', 'so', 'stupid', 'worthless', 'him', 'maybe', 'problems', 'help', 'dalmation', 'ate', 'buying', 'has', 'my', 'licks', 'to', 'garbage', 'how', 'love', 'cute', 'posting', 'mr', 'food', 'flea', 'stop', 'steak', 'take', 'is']


## 把句子转换为词向量
使用list

In [5]:
def setOfWordVec(wordVec, words):
    N = len(wordVec)
    res = [0]*N
    for word in words:
        if word in wordVec:
            res[wordVec.index(word)] = 1
    return res

In [6]:
test_words = ['my', 'dog', 'is', 'quit']
test_res = setOfWordVec(wordVec, test_words)
index = list(range(len(wordVec)))
print(wordVec)
print(index)
print(test_res)

['dog', 'please', 'I', 'park', 'quit', 'not', 'so', 'stupid', 'worthless', 'him', 'maybe', 'problems', 'help', 'dalmation', 'ate', 'buying', 'has', 'my', 'licks', 'to', 'garbage', 'how', 'love', 'cute', 'posting', 'mr', 'food', 'flea', 'stop', 'steak', 'take', 'is']
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]
[1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]


## 训练函数

In [7]:
def trainNBS(train_data, train_lables, wordVec):
    ''' set of train labels '''
    label_v = list(set(train_lables))
    label_N = len(label_v)
    vec_len = len(wordVec)
    Ps = np.zeros((label_N, vec_len))
    Ns = np.zeros((label_N, 1))
    for data, label in zip(train_data, train_lables):
        vec = setOfWordVec(wordVec, data)
        idx = label_v.index(label)
        Ps[idx] += np.array(vec)
        Ns[idx] += 1
    return Ps/Ns, Ns.squeeze()/len(train_lables)

In [8]:
train_Pxy, train_Py = trainNBS(dataset, datalabel, wordVec)

In [9]:
print("x|y\n", train_Pxy)
print("y\n", train_Py)

x|y
 [[0.33333333 0.33333333 0.33333333 0.         0.         0.
  0.33333333 0.         0.         0.66666667 0.         0.33333333
  0.33333333 0.33333333 0.33333333 0.         0.33333333 1.
  0.33333333 0.33333333 0.         0.33333333 0.33333333 0.33333333
  0.         0.33333333 0.         0.33333333 0.33333333 0.33333333
  0.         0.33333333]
 [0.66666667 0.         0.         0.33333333 0.33333333 0.33333333
  0.         1.         0.66666667 0.33333333 0.33333333 0.
  0.         0.         0.         0.33333333 0.         0.
  0.         0.33333333 0.33333333 0.         0.         0.
  0.33333333 0.         0.33333333 0.         0.33333333 0.
  0.33333333 0.        ]]
y
 [0.5 0.5]


In [10]:
def classifyNBS(testDoc, Pxy, Py, wordVec):
    vec = setOfWordVec(wordVec, testDoc)
    mat = np.dot([vec], Pxy.T) * Py.reshape(1, -1)
    mat = mat.squeeze()
    print(mat)
    idx = np.argmax(mat)
    return idx

In [11]:
''' test naive bayes '''
test_words = ['love', 'my', 'dalmation']
print(classifyNBS(test_words, train_Pxy, train_Py, wordVec))
test_words = ['I', 'stupid']
print(classifyNBS(test_words, train_Pxy, train_Py, wordVec))

[0.83333333 0.        ]
0
[0.16666667 0.5       ]
1


## 使用词袋模型

In [12]:
def bagOfWordVec(wordVec, words):
    N = len(wordVec)
    res = [0]*N
    for word in words:
        if word in wordVec:
            res[wordVec.index(word)] += 1
    return res

## 对样本字词进行拆解

In [14]:
def text_split(text):
    import re
    token_list = re.split(r'\W*', text)
    return [token.lower() for token in token_list if len(token) > 2]

In [52]:
def load_dataset(pos_dir, neg_dir):
    import os
    
    labels = []
    dataset = []
    
    pos_files = os.listdir(pos_dir)
    neg_files = os.listdir(neg_dir)
    
    for file in pos_files:
        with open(os.path.join(pos_dir, file), 'r', encoding='ISO-8859-1') as fp:
            words = []
            line = fp.read()
            words.extend(text_split(line))
            dataset.append(words)
            labels.append(1)
    
    for file in neg_files:
        with open(os.path.join(neg_dir, file), 'r', encoding='ISO-8859-1') as fp:
            words = []
            line = fp.read()
            words.extend(text_split(line))
            dataset.append(words)
            labels.append(0)
    
    return dataset, labels

In [57]:
pos_dir = './email/ham'
neg_dir = './email/spam'
dataset, labels = load_dataset(pos_dir, neg_dir)

print(dataset[:10])
print(labels[:10])

[['what', 'going', 'there', 'talked', 'john', 'email', 'talked', 'about', 'some', 'computer', 'stuff', 'that', 'went', 'bike', 'riding', 'the', 'rain', 'was', 'not', 'that', 'cold', 'went', 'the', 'museum', 'yesterday', 'was', 'get', 'and', 'they', 'had', 'free', 'food', 'the', 'same', 'time', 'was', 'giants', 'game', 'when', 'got', 'done', 'had', 'take', 'the', 'train', 'with', 'all', 'the', 'giants', 'fans', 'they', 'are', 'drunk'], ['saw', 'this', 'the', 'way', 'the', 'coast', 'thought', 'might', 'like', 'hangzhou', 'huge', 'one', 'day', 'wasn', 'enough', 'but', 'got', 'glimpse', 'went', 'inside', 'the', 'china', 'pavilion', 'expo', 'pretty', 'interesting', 'each', 'province', 'has', 'exhibit'], ['peter', 'sure', 'thing', 'sounds', 'good', 'let', 'know', 'what', 'time', 'would', 'good', 'for', 'you', 'will', 'come', 'prepared', 'with', 'some', 'ideas', 'and', 'can', 'from', 'there', 'regards', 'vivek'], ['this', 'mail', 'was', 'sent', 'from', 'notification', 'only', 'address', 'that

/home/ubuntu/anaconda3/lib/python3.6/re.py:212: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [67]:
'''
shuffled the dataset and labels
'''
index = list(range(len(labels)))
random.shuffle(index)

print(index)
dataset_new = []
labels_new = []
for i in index:
    dataset_new.append(dataset[i])
    labels_new.append(labels[i])

print(dataset_new[:5])
print(labels_new[:5])

[4, 28, 30, 17, 43, 24, 16, 10, 26, 42, 49, 2, 0, 14, 9, 47, 33, 29, 7, 32, 1, 38, 45, 48, 25, 35, 8, 40, 36, 31, 18, 46, 23, 15, 39, 20, 41, 11, 37, 44, 13, 22, 3, 34, 6, 27, 19, 12, 21, 5]
[['will', 'there', 'the', 'latest'], ['you', 'have', 'everything', 'gain', 'incredib1e', 'gains', 'length', 'inches', 'yourpenis', 'permanantly', 'amazing', 'increase', 'thickness', 'yourpenis', 'betterejacu1ation', 'control', 'experience', 'rock', 'harderecetions', 'explosive', 'intenseorgasns', 'increase', 'volume', 'ofejacu1ate', 'doctor', 'designed', 'and', 'endorsed', '100', 'herbal', '100', 'natural', '100', 'safe'], ['you', 'have', 'everything', 'gain', 'incredib1e', 'gains', 'length', 'inches', 'yourpenis', 'permanantly', 'amazing', 'increase', 'thickness', 'yourpenis', 'betterejacu1ation', 'control', 'experience', 'rock', 'harderecetions', 'explosive', 'intenseorgasns', 'increase', 'volume', 'ofejacu1ate', 'doctor', 'designed', 'and', 'endorsed', '100', 'herbal', '100', 'natural', '100', '